In [2]:
import numpy as np, numpy.random as npr, random as r
import tensorflow as tf  
from NavTask import NavigationTask
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.slim as slim

In [3]:
#### Node --> Children  

# From: https://github.com/ericjang/gumbel-softmax
class GumbelSoftmax(object):
    def sample_gumbel(shape, eps=1e-20): 
      """Sample from Gumbel(0, 1)"""
      """  dd """
      U = tf.random_uniform(shape,minval=0,maxval=1)
      return -tf.log(-tf.log(U + eps) + eps)

    def gumbel_softmax_sample(logits, temperature): 
      """ Draw a sample from the Gumbel-Softmax distribution"""
      y = logits + GumbelSoftmax.sample_gumbel(tf.shape(logits))
      return tf.nn.softmax( y / temperature)

    def gumbel_softmax(logits, temperature, hard=False):
      """Sample from the Gumbel-Softmax distribution and optionally discretize.
      Args:
        logits: [batch_size, n_class] unnormalized log-probs
        temperature: non-negative scalar
        hard: if True, take argmax, but differentiate w.r.t. soft sample y
      Returns:
        [batch_size, n_class] sample from the Gumbel-Softmax distribution.
        If hard=True, then the returned sample will be one-hot, otherwise it will
        be a probabilitiy distribution that sums to 1 across classes
      """
      y = GumbelSoftmax.gumbel_softmax_sample(logits, temperature)
      if hard:
        k = tf.shape(logits)[-1]
        #y_hard = tf.cast(tf.one_hot(tf.argmax(y,1),k), y.dtype)
        y_hard = tf.cast(tf.equal(y,tf.reduce_max(y,1,keep_dims=True)),y.dtype)
        y = tf.stop_gradient(y_hard - y) + y
      return y

#sess = tf.Session()
#sess.run(GumbelSoftmax.gumbel_softmax(tf.constant([[0.5, 0.5]]), 0.5, hard=True))

In [6]:
class SimulationPolicy(object):
    
    def __init__(self, obs_space, act_space, h_size=100):
        print("Observation Space: " , obs_space)
        print("Action Space: ", act_space)
        self.h_size = h_size
        # Input space: [Episode_length, observations], output:[Episode_Length,action_space]
        self.input = tf.placeholder(tf.float32, [None] + list(obs_space))
        self.act_space = act_space
        #self.output = self.getSoftAction(self.input)
        self.sampleOutput = self.sample(self.input)
        self.trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, tf.get_variable_scope().name) 
    
    def getSoftAction(self,observation, reuse=None):
        with tf.variable_scope("softaction", reuse=reuse):
            hidden = slim.fully_connected(observation, self.h_size, biases_initializer=None, activation_fn=tf.nn.relu)
            return slim.fully_connected(hidden, self.act_space, activation_fn=tf.nn.softmax, biases_initializer=None)
        
    def sample(self,observation,temperature=0.5, reuse=None):
        sess = tf.get_default_session()
        #print(sess.run(tf.report_uninitialized_variables()))
        softAction = self.getSoftAction(observation, reuse=reuse)
        #sess.run(tf.global_variables_initializer())
        return GumbelSoftmax.gumbel_softmax(softAction, temperature, hard=True)
        

In [8]:
# sess = tf.Session()
with tf.Graph().as_default(), tf.Session() as sess:
    simpolicy = SimulationPolicy( np.shape([1,1]),3)
    sess.run(tf.global_variables_initializer())
    
    sample = simpolicy.sample(tf.constant([[5.0,1.0]]), reuse=True) 
    sample1 = simpolicy.sample(tf.constant([[5.0,1.0]]), reuse=True)
    
    writer = tf.summary.FileWriter('logs', sess.graph)
    x, _ = sess.run([sample, sample1])
    #writer.close()
    print(x)

Observation Space:  (2,)
Action Space:  3
[[ 1.  0.  0.]]


In [ ]:
class Node(object):
    
    def __init__(self, parent_node):
        self.parent = parent_node
        self.children = []
        
    def addChild(self,forwardModel,simPolicy):
        raw_actio_probs = simPolicy
        

class Tree(object):
    
    def __init__(self,):
        